In [54]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et
from shutil import move

In [11]:
xml_list=glob("C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/*.xml")
xml_list=list(map(lambda x: x.replace('\\','/'),xml_list))

In [12]:
xml_list

['C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/01.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/02.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/03.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/04.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/05.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/06.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/07.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/08.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/09.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/10.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/11.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/12.xml',
 'C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/13.xml',
 'C:/Users/D

In [20]:
def extract_text(filename):
    tree=et.parse(filename)
    root=tree.getroot()
    
    #filename
    image_name=root.find('filename').text
    
    #size
    width=root.find('size').find('width').text
    height=root.find('size').find('height').text
    
    #objects
    objs=root.findall('object')
    parser=[]
    for obj in objs:
        name=obj.find('name').text
        bndbox=obj.find('bndbox')
        xmin=bndbox.find('xmin').text
        ymin=bndbox.find('ymin').text
        xmax=bndbox.find('xmax').text
        ymax=bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,ymin,xmax,ymax])
    return parser

In [22]:
parser_all=list(map(extract_text,xml_list))

In [24]:
data=reduce(lambda x,y:x+y,parser_all)

In [26]:
df=pd.DataFrame(data,columns=['filename','width','height','name','xmin','ymin','xmax','ymax'])

In [27]:
df

,filename,width,height,name,xmin,ymin,xmax,ymax
0,01.jpg,612,383,crater,187,210,246,271
1,01.jpg,612,383,crater,318,164,364,208
2,01.jpg,612,383,crater,294,80,346,131
3,01.jpg,612,383,crater,377,108,405,137
4,01.jpg,612,383,crater,457,141,510,197
...,...,...,...,...,...,...,...,...
77,21.jpg,612,242,boulder,360,128,394,157
78,21.jpg,612,242,boulder,379,181,427,210
79,22.jpg,1110,1044,boulder,416,371,720,658
80,22.jpg,1110,1044,boulder,219,108,367,194


In [33]:
df['name'].value_counts()

name
crater     70
boulder    12
Name: count, dtype: int64

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  82 non-null     object
 1   width     82 non-null     object
 2   height    82 non-null     object
 3   name      82 non-null     object
 4   xmin      82 non-null     object
 5   ymin      82 non-null     object
 6   xmax      82 non-null     object
 7   ymax      82 non-null     object
dtypes: object(8)
memory usage: 5.2+ KB


In [35]:
cols=['width','height','xmin','ymin','xmax','ymax']
df[cols]=df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  82 non-null     object
 1   width     82 non-null     int64 
 2   height    82 non-null     int64 
 3   name      82 non-null     object
 4   xmin      82 non-null     int64 
 5   ymin      82 non-null     int64 
 6   xmax      82 non-null     int64 
 7   ymax      82 non-null     int64 
dtypes: int64(6), object(2)
memory usage: 5.2+ KB


In [36]:
df['center_x']=((df['xmin']+df['xmax'])/2)/df['width']
df['center_y']=((df['ymin']+df['ymax'])/2)/df['height']

df['w']=(df['xmax']-df['xmin'])/df['width']
df['h']=(df['ymax']-df['ymin'])/df['height']

In [37]:
df

,filename,width,height,name,xmin,ymin,xmax,ymax,center_x,center_y,w,h
0,01.jpg,612,383,crater,187,210,246,271,0.353758,0.627937,0.096405,0.159269
1,01.jpg,612,383,crater,318,164,364,208,0.557190,0.485640,0.075163,0.114883
2,01.jpg,612,383,crater,294,80,346,131,0.522876,0.275457,0.084967,0.133159
3,01.jpg,612,383,crater,377,108,405,137,0.638889,0.319843,0.045752,0.075718
4,01.jpg,612,383,crater,457,141,510,197,0.790033,0.441253,0.086601,0.146214
...,...,...,...,...,...,...,...,...,...,...,...,...
77,21.jpg,612,242,boulder,360,128,394,157,0.616013,0.588843,0.055556,0.119835
78,21.jpg,612,242,boulder,379,181,427,210,0.658497,0.807851,0.078431,0.119835
79,22.jpg,1110,1044,boulder,416,371,720,658,0.511712,0.492816,0.273874,0.274904
80,22.jpg,1110,1044,boulder,219,108,367,194,0.263964,0.144636,0.133333,0.082375


In [38]:
images=df['filename'].unique()

In [41]:
len(images)

22

In [42]:
# 80% train and 20% test
img_df=pd.DataFrame(images,columns=['filename'])
img_train=tuple(img_df.sample(frac=0.8)['filename'])
img_test=tuple(img_df.query(f'filename not in {img_train}')['filename'])

In [43]:
train_df=df.query(f'filename in {img_train}')
test_df=df.query(f'filename in {img_test}')

In [44]:
train_df

,filename,width,height,name,xmin,ymin,xmax,ymax,center_x,center_y,w,h
0,01.jpg,612,383,crater,187,210,246,271,0.353758,0.627937,0.096405,0.159269
1,01.jpg,612,383,crater,318,164,364,208,0.557190,0.485640,0.075163,0.114883
2,01.jpg,612,383,crater,294,80,346,131,0.522876,0.275457,0.084967,0.133159
3,01.jpg,612,383,crater,377,108,405,137,0.638889,0.319843,0.045752,0.075718
4,01.jpg,612,383,crater,457,141,510,197,0.790033,0.441253,0.086601,0.146214
...,...,...,...,...,...,...,...,...,...,...,...,...
75,19.jpg,800,449,boulder,125,175,532,373,0.410625,0.610245,0.508750,0.440980
76,20.jpg,663,662,boulder,182,199,538,397,0.542986,0.450151,0.536953,0.299094
79,22.jpg,1110,1044,boulder,416,371,720,658,0.511712,0.492816,0.273874,0.274904
80,22.jpg,1110,1044,boulder,219,108,367,194,0.263964,0.144636,0.133333,0.082375


In [45]:
test_df

,filename,width,height,name,xmin,ymin,xmax,ymax,center_x,center_y,w,h
14,03.png,1200,866,crater,307,231,836,748,0.476250,0.565242,0.440833,0.596998
15,03.png,1200,866,crater,1102,682,1196,763,0.957500,0.834296,0.078333,0.093533
27,06.jpg,612,383,crater,202,249,464,357,0.544118,0.791123,0.428105,0.281984
59,15.jpg,612,408,crater,31,262,166,292,0.160948,0.678922,0.220588,0.073529
60,15.jpg,612,408,crater,132,295,282,333,0.338235,0.769608,0.245098,0.093137
77,21.jpg,612,242,boulder,360,128,394,157,0.616013,0.588843,0.055556,0.119835
78,21.jpg,612,242,boulder,379,181,427,210,0.658497,0.807851,0.078431,0.119835


In [51]:
def label_encoding(x):
    labels={'crater':0,'boulder':1}
    return labels[x]

In [52]:
train_df['id']=train_df['name'].apply(label_encoding)
test_df['id']=test_df['name'].apply(label_encoding)

C:\Users\DELL\AppData\Local\Temp\ipykernel_7868\1049210348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id']=train_df['name'].apply(label_encoding)
C:\Users\DELL\AppData\Local\Temp\ipykernel_7868\1049210348.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id']=test_df['name'].apply(label_encoding)


In [53]:
train_df

,filename,width,height,name,xmin,ymin,xmax,ymax,center_x,center_y,w,h,id
0,01.jpg,612,383,crater,187,210,246,271,0.353758,0.627937,0.096405,0.159269,0
1,01.jpg,612,383,crater,318,164,364,208,0.557190,0.485640,0.075163,0.114883,0
2,01.jpg,612,383,crater,294,80,346,131,0.522876,0.275457,0.084967,0.133159,0
3,01.jpg,612,383,crater,377,108,405,137,0.638889,0.319843,0.045752,0.075718,0
4,01.jpg,612,383,crater,457,141,510,197,0.790033,0.441253,0.086601,0.146214,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,19.jpg,800,449,boulder,125,175,532,373,0.410625,0.610245,0.508750,0.440980,1
76,20.jpg,663,662,boulder,182,199,538,397,0.542986,0.450151,0.536953,0.299094,1
79,22.jpg,1110,1044,boulder,416,371,720,658,0.511712,0.492816,0.273874,0.274904,1
80,22.jpg,1110,1044,boulder,219,108,367,194,0.263964,0.144636,0.133333,0.082375,1


In [58]:
test_folder='C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/test'
os.mkdir(test_folder)

In [60]:
train_folder='C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/train'
os.mkdir(train_folder)

In [61]:
cols=['filename','id','center_x','center_y','w','h']
groupby_obj_train=train_df[cols].groupby('filename')
groupby_obj_test=test_df[cols].groupby('filename')

In [71]:
def save_data(filename,folder_path,group_obj):
    src='C:/Users/DELL/OneDrive/Desktop/yolo/data_preparation/data_images/'+filename
    dst=os.path.join(folder_path,filename)
    move(src,dst)
    
    text_filename=os.path.join(folder_path,os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)

In [72]:
filename_series=pd.Series(groupby_obj_train.groups.keys())

In [74]:
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
dtype: object

In [75]:
filename_series_test=pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

0    None
1    None
2    None
3    None
dtype: object